<a href="https://colab.research.google.com/github/MdelaVilla/MORS/blob/main/Clasificaci%C3%B3n_binaria_con_Transformers_y_corpus_HateEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Importar librerías**

In [1]:
!pip install datasets transformers
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
import re
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **2. Cargar Dataset**

In [69]:
# Cargamos el dataset desde un archivo CSV
df = pd.read_csv('/content/drive/MyDrive/corpus/HateEval/train_es.tsv', sep='\t', usecols =["text","HS"])
# Mostramos los primeros registros para verificar que los datos se cargaron correctamente
print(df.head())

                                                text  HS
0  Easyjet quiere duplicar el número de mujeres p...   1
1  El gobierno debe crear un control estricto de ...   1
2  Yo veo a mujeres destruidas por acoso laboral ...   0
3  — Yo soy respetuoso con los demás, sólamente l...   0
4  Antonio Caballero y como ser de mal gusto e ig...   0


In [70]:
df.columns = ['text', 'label']
df

,text,label
0,Easyjet quiere duplicar el número de mujeres p...,1
1,El gobierno debe crear un control estricto de ...,1
2,Yo veo a mujeres destruidas por acoso laboral ...,0
3,"— Yo soy respetuoso con los demás, sólamente l...",0
4,Antonio Caballero y como ser de mal gusto e ig...,0
...,...,...
4464,@miriaan_ac @Linaveso_2105 @HumildesSquad_ CÁL...,1
4465,"@IvanDuque presidente en Cúcuta , tenemos prob...",1
4466,- Callaté Visto Que Te Dejo En Puta🎤🎶,0
4467,-¿porque los hombres se casan con las mujeres?...,1


In [71]:
#Vemos cuantas instancias hay de cada tipo en el dataset

value_counts = df['label'].value_counts()
value_counts

,count
label,
0,2631
1,1838


## **3. Preprocesado básico**

In [72]:
def to_lowercase(tweet):
    """Takes a string and converts all characters to lowercase"""
    return tweet.lower()

In [73]:
# Aplicamos el preprocesamiento al texto de nuestro DataFrame
df['text'] = df['text'].apply(to_lowercase)

Comprobamos si ha funcionado el preprocesado

In [74]:
#Mostramos la fila 4091 y 165

print(df.iloc[20]['text'])
print(df.iloc[80]['text'])

mira de vd estoy hasta la polla ya, no aguanto más tonterías
me obligan a tragar polla y me mean en la cara 😮 https://t.co/8shynivgxa


## **4. Conversión a Dataset y Tokenización**

Usamos el tokenizador de BERT para convertir el texto en vectores que pueda entender el modelo, truncando o rellenando hasta la longitud máxima establecida.

In [75]:
# Cargar el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convertir el DataFrame a Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

# Definir la función de tokenización
def tokenize_function(example):
    return tokenizer(example["text"], padding='max_length', truncation=True, max_length=128)

# Aplicar la tokenización al Dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Convertir las columnas necesarias a formato tensor de PyTorch
tokenized_dataset = tokenized_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])

# Verificar el dataset tokenizado
print(tokenized_dataset[0])

Map:   0%|          | 0/4469 [00:00<?, ? examples/s]

{'label': tensor(1), 'input_ids': tensor([  101,  3733, 15759, 21864,  7869,  4241, 24759,  5555,  2099,  3449,
        16371,  5017,  2080,  2139, 14163, 20009,  2229,  4405,  2080,  1005,
        12297,  2015, 10722, 11498,  9706,  2906, 10010,  3449, 20704,  3258,
         1012,  1012,  8299,  1024,  1013,  1013,  1056,  1012,  2522,  1013,
         4805, 11231, 13687,  2213,  2692,  2683,  2595,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

## **5. División en Conjuntos de Entrenamiento, validación y test**

Dividimos el conjunto de entrenamiento en 65% train, 15% validación y 20% test

In [76]:
# Dividir en entrenamiento (80%) y test (20%)
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_data = train_test_split['train']
test_data = train_test_split['test']


# Dividir el conjunto de entrenamiento en entrenamiento (70%) y validación (30%)
train_valid_split = train_data.train_test_split(test_size=0.3, seed=42)
train_data = train_valid_split['train']
valid_data = train_valid_split['test']

In [77]:
train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2502
})

In [78]:
valid_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1073
})

In [79]:
test_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 894
})

## **6. Carga del modelo BERT para clasificación**

Cargamos el modelo preentrenado BertForSequenceClassification con dos etiquetas de salida.

In [80]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## **7. Configuración de parámetros de entrenamiento**

Configuramos los parámetros de entrenamiento, incluyendo el número de épocas y el tamaño del lote.

In [81]:
training_args = TrainingArguments(
    output_dir='./results',       # Carpeta para guardar los resultados
    num_train_epochs=4,           # Número de épocas
    per_device_train_batch_size=8,  # Tamaño del lote de entrenamiento
    per_device_eval_batch_size=8,  # Tamaño del lote de evaluación
    evaluation_strategy="epoch",   # Evaluar al final de cada época
    logging_dir='./logs',
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
    save_strategy = 'epoch'
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## **8. Entrenamiento y evaluación del modelo**

Usamos Trainer para entrenar y evaluar el modelo con el conjunto de entrenamiento y el de validación.

In [82]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Definir la función compute_metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=-1)

    # Calcular las métricas usando las predicciones y etiquetas
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }




trainer = Trainer(
    model=model,                         # Modelo a entrenar
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=train_data,            # Conjunto de entrenamiento
    eval_dataset=valid_data,             # Conjunto de evaluación
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

# Ejecutar el entrenamiento
trainer.train()

# Evaluar el modelo
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.665633,0.602050,0.525692,0.302273,0.383838
2,0.691600,0.676923,0.589935,0.000000,0.000000,0.000000
3,0.691600,0.677085,0.589935,0.000000,0.000000,0.000000
4,0.687200,0.676953,0.589935,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.6656332612037659,
 'eval_accuracy': 0.6020503261882573,
 'eval_precision': 0.525691699604743,
 'eval_recall': 0.30227272727272725,
 'eval_f1': 0.3838383838383838,
 'eval_runtime': 7.4374,
 'eval_samples_per_second': 144.271,
 'eval_steps_per_second': 18.152,
 'epoch': 4.0}

## **9. Prueba del modelo en nuevos textos**

Ahora podemos hacer predicciones en nuevos textos para evaluar el modelo.

In [83]:
# Evaluate the model on the test set and print the classification report

test_results = trainer.predict(test_data)
predictions = test_results.predictions.argmax(axis=-1)
labels = test_results.label_ids

print(predictions)
#print(labels)

print(classification_report(labels, predictions))

[0 1 1 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1
 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0
 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1
 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1
 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 